# Простая (однослойная) рекуррентная сеть

In [ ]:
import os

# использованием PyTorch как фреймворка
os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras
from keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandas_datareader.data as web

# Нормировка данных
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler

In [ ]:
# Подготовка датасета для обучения в формате inputs, outputs
def make_datasets(input_data, n_inputs=2, n_outputs=1, gap=0):
	L = len(input_data)
	y = np.full((L-n_inputs-n_outputs-gap, n_outputs), 0.0)
	X = np.full((L-n_inputs-n_outputs-gap, n_inputs), 0.0)

	for i in range(n_inputs):
		X[:,i] = input_data[i:L-n_inputs-n_outputs-gap+i]

	for i in range(n_outputs):
		y[:,i] = input_data[n_inputs+gap+i:L-n_outputs+i]

	return X, y

In [ ]:
rate = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01')
len(rate)

In [ ]:
rate.plot()
plt.show()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Переведем ряда массив Numpy
series = rate.iloc[:,0].values

# scaler = MinMaxScaler()
# series = scaler.fit_predict(series)

# Модель №1: многошаговый прогноз

In [ ]:
# задаём ширину окна и горизонт прогнозирования
n_lags, fh= 20, 5

X, y = make_datasets(series, n_inputs=n_lags, n_outputs=fh)

Зададим простую рекуррентную с простым выходным слоем

$$
\begin{aligned}
	h_t&=f_{rnn}(W_xx_t+W_h h_{t-1}+b_h) & h_t&\in\mathbb{R}^{units} \\
	y_t&=Wh_t+b_y & y_t&\in\mathbb{R}^{fh}
\end{aligned}
$$

In [ ]:
# Инициализируем модель
model = keras.Sequential().to(device)
# Добавляем входной слой и размер батча
model.add(keras.Input(shape=(n_lags,1), batch_size=32))
# Добавляем слой RNN
model.add(layers.SimpleRNN(units=50, activation='relu', seed=0))
# Добавляем слой LSTM
# model.add(layers.LSTM(units=50, activation='relu', seed=0))
# Добавляем слой GRU
# model.add(layers.GRU(units=50, activation='relu', seed=0))
# Задаём выходной слой
model.add(layers.Dense(fh))
# Собираем модель
model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
# Можно задать для реплицируемои
# keras.utils.set_random_seed(42)

# Обучение модели
history = model.fit(
    X, y,
    epochs=10,
    batch_size=None,
    shuffle=False,
    verbose=1
)

In [ ]:
# Прогнозирование
y_pred = model.predict(np.reshape(series[-n_lags:], (1, n_lags)))[0]
# если данные были нормированы, то применим обратное преобразование
# y_pred = scaler.inverse_transform(y_pred)
y_pred

In [ ]:
last_obs = 50
plt.plot(np.arange(len(series)-last_obs, len(series)), series[-last_obs:])
plt.plot(np.arange(len(series), len(series)+fh), y_pred)
plt.show()

# Модель №2: последовательный многошаговый прогноз

In [ ]:
# задаём ширину окна и горизонт прогнозирования
n_lags, fh= 20, 1

X, y = make_datasets(series, n_inputs=n_lags, n_outputs=fh)

In [ ]:
# Инициализируем модель
model = keras.Sequential().to(device)
# Добавляем входной слой и размер батча
model.add(keras.Input(shape=(n_lags,1), batch_size=32))
# Добавляем слой RNN
model.add(layers.SimpleRNN(units=50, activation='relu', seed=0))
# Добавляем слой LSTM
# model.add(layers.LSTM(units=50, activation='relu', seed=0))
# Добавляем слой GRU
# model.add(layers.GRU(units=50, activation='relu', seed=0))
# Задаём выходной слой
model.add(layers.Dense(fh))
# Собираем модель
model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
# Можно задать для реплицируемои
# keras.utils.set_random_seed(42)

# Обучение модели
history = model.fit(
    X, y,
    epochs=10,
    batch_size=None,
    shuffle=False,
    verbose=1
)

In [ ]:
# Прогнозирование через скользящее окно
steps = 10
forecasts = np.array([])
sliding_window = np.reshape(series[-n_lags:], (1, n_lags))

for _ in range(steps):
	y_pred = model.predict(sliding_window)
	forecasts = np.concatenate([forecasts, y_pred[0]])
	sliding_window = np.hstack([sliding_window[:,1:], y_pred])

forecasts = np.array(forecasts)

In [ ]:
last_obs = 50
plt.plot(np.arange(len(series)-last_obs, len(series)), series[-last_obs:])
plt.plot(np.arange(len(series), len(series)+fh), forecasts)
plt.show()